# 7. Статистическая проверка гипотез для связанных выборок. A/B тесты

In [ ]:
import math
import seaborn as sns
import scipy.stats as st
from scipy.stats import wilcoxon
from scipy.stats import mannwhitneyu
from scipy.stats import f_oneway
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#### Описание набора данных
Профессор Оук скопировал все содержимое память одного устройства Pokedex, в результате чего получился набор данных, с которым Вы будете работать в этой задаче. В этом файле каждая строка представляет характеристики одного покемона:

* `pid`: Numeric - ID покемона
* `HP`: Numeric - Очки здоровья
* `Attack`: Numeric - Сила обычной атаки
* `Defense`: Numeric - Сила обычной защиты
* `Sp. Atk`: Numeric - Сила специальной атаки
* `Sp. Def`: Numeric - Сила специальной защиты
* `Speed`: Numeric - Скорость движений
* `Legendary`: Boolean - «True», если покемон редкий
* `Class 1`: Categorical - Класс покемона
* `Class 2`: Categorical - Класс покемона

In [ ]:
pokemon_path = 'https://raw.githubusercontent.com/a-milenkin/datasets_for_t-tests/main/pokemon.csv'
pokemon = pd.read_csv(pokemon_path, error_bad_lines=False)  # Откроем датасет
pokemon.head()

 
#Обратите внимание, что у покемона может быть один или два класса. Если у покемона два класса, считается,
#что они имеют одинаковую значимость.

### Задание № 1 (Обязательно):
Профессор Оук подозревает, что покемоны в классе grass имеют более сильную обычную атаку, чем у покемонов в классе rock. Проверьте, прав ли он, и убедите его в своем выводе статистически.

Примечание: если есть покемоны, которые относятся к обоим классам, просто выбросьте их;

Вы можете предположить, что распределение обычных атак является нормальным для всех классов покемонов.

In [ ]:
df = pokemon.copy()
df = df.drop(['pid'], axis = 1).drop(['HP'], axis = 1).drop(['Defense'], axis = 1).drop(['Sp. Atk'], axis = 1).drop(['Sp. Def'], axis = 1).drop(['Speed'], axis = 1).drop(['Legendary'], axis = 1)
df = df.drop(df.loc[(df['Class 1'] + df['Class 2']).isin(['GrassRock', 'RockGrass'])].index)
df = df.loc[df['Class 1'].isin(['Grass', 'Rock']) | df['Class 2'].isin(['Grass', 'Rock'])]

df

In [ ]:
grass = df[(df['Class 1'] == 'Grass') | (df['Class 2'] == 'Grass')]
rock = df[(df['Class 1'] == 'Rock') | (df['Class 2'] == 'Rock')]

можно посмотреть, есть ли различия визуально:

In [ ]:
sns.boxplot(grass['Attack'])

In [ ]:
b = sns.boxplot(rock['Attack'])

на графике видно, что средняя сила атаки у покемонов в клаасе rock все же выше

In [ ]:
t, p = st.ttest_ind(grass['Attack'], rock['Attack'], equal_var=False)
print(t, p)

выше проверили на стат значимость выборки

*я пыталась проверить по критерию Уилкоксона, но лучше по Манна-Уитни так как на Уилкоксона всё ругается

In [ ]:
#H0 - нет стат значимых различий
#H1 - есть стат значимые различия

stat, p = wilcoxon(grass['Attack'], rock['Attack'])
print('Statistics=%.3f, p=%.3f' % (stat, p))

alpha = 0.05
if p > alpha:
    print('нет стат значимых различий (не отвергаем H0)')
else:
    print('есть стат значимые различия (отвергаем H0)')

In [ ]:
len(grass['Attack']), len(rock['Attack'])

In [ ]:
#H0 - нет стат значимых различий
#H1 - есть стат значимые различия

stat, p = mannwhitneyu(grass['Attack'], rock['Attack'])
print('Statistics=%.3f, p=%.3f' % (stat, p))

alpha = 0.05
if p > alpha:
    print('нет стат значимых различий (не отвергаем H0)')
else:
    print('есть стат значимые различия (отвергаем H0)')

и еще раз проверили. все окей, дальше можем сравнивать показатели атаки у покемонов

In [ ]:
if grass['Attack'].mean() > rock['Attack'].mean():
  print('Подозрения профессора Оука оправданы')

else:
  print('Подозрения профессора Оука не оправданы')

### Задание № 2 (Обязательно):
Профессор Оук уже долго не может спать по ночам, ведь его волнует вопрос: а правда, что покемоны в классе Water в среднем быстрее покемонов в классе Normal.

Проверьте, прав ли он, и убедите его в своем выводе статистически.

Примечание: если есть покемоны, которые относятся к обоим классам, выбросьте их;

Вы можете предположить, что распределение скорости движения является нормальным для всех классов покемонов.

In [ ]:
df_1 = pokemon.copy()
df_1 = df_1.drop(['pid'], axis = 1).drop(['HP'], axis = 1).drop(['Defense'], axis = 1).drop(['Sp. Atk'], axis = 1).drop(['Sp. Def'], axis = 1).drop(['Attack'], axis = 1).drop(['Legendary'], axis = 1)
df_1 = df_1.drop(df.loc[(df_1['Class 1'] + df_1['Class 2']).isin(['WaterNormal', 'NormalWater'])].index)
df_1 = df_1.loc[df_1['Class 1'].isin(['Water', 'Normal']) | df_1['Class 2'].isin(['Water', 'Normal'])]

df_1

In [ ]:
water = df_1[(df_1['Class 1'] == 'Water') | (df_1['Class 2'] == 'Water')]
normal = df_1[(df_1['Class 1'] == 'Normal') | (df_1['Class 2'] == 'Normal')]

смотрим различия визуально:

In [ ]:
sns.boxplot(water['Speed'])

In [ ]:
sns.boxplot(normal['Speed'])

различия видно, но они прям минимальные


далее проверяем, различаются ли статистически наши выборки. По условию распределение нормальное, поэтому t-test:

In [ ]:
t, p = st.ttest_ind(water['Speed'], normal['Speed'], equal_var=False)
print(t, p)

но на нормальность мы ничего не проверяли, поэтому бахнем тест Манна-Уитни так как выборки независимые

In [ ]:
#H0 - нет стат значимых различий
#H1 - есть стат значимые различия

stat, p = mannwhitneyu(water['Speed'], normal['Speed'])
print('Statistics=%.3f, p=%.3f' % (stat, p))

alpha = 0.05
if p > alpha:
    print('нет стат значимых различий (не отвергаем H0)')
else:
    print('есть стат значимые различия (отвергаем H0)')

In [ ]:
if water['Speed'].mean() > normal['Speed'].mean():
  print('Покемоны в классе Water в среднем быстрее')

else:
  print('Покемоны в классе Normal в среднем быстрее')

### Задание № 3:
Профессор Оук тот еще безумец. Он изобрел сыворотку, способную ускорить покемона. Однако, мы усомнились в эффективности его вакцины. Профессор дал эту сыворотку следующим покемонам: смотри массив treathed_pokemon. 

Проверьте, работает ли вообще его сыворотка, убедите всех в своем выводе статистически.

Вы можете предположить, что распределение скорости движения является нормальным для всех классов покемонов.

In [ ]:
# Покемоны, которые принимали сыворотку увеличения скорости
treathed_pokemon = ['Mega Beedrill','Mega Alakazam', 'Deoxys Normal Forme', 'Mega Lopunny']

In [ ]:
fast = pokemon[pokemon['Name'].isin(['Mega Beedrill','Mega Alakazam', 'Deoxys Normal Forme', 'Mega Lopunny'])]
regular = pokemon[~pokemon['Name'].isin(['Mega Beedrill','Mega Alakazam', 'Deoxys Normal Forme', 'Mega Lopunny'])]

In [ ]:
pokemon.info(), fast.info(), regular.info()

In [ ]:
fast = fast.drop(['pid'], axis = 1).drop(['HP'], axis = 1).drop(['Defense'], axis = 1).drop(['Sp. Atk'], axis = 1).drop(['Sp. Def'], axis = 1).drop(['Attack'], axis = 1).drop(['Legendary'], axis = 1)
regular = regular.drop(['pid'], axis = 1).drop(['HP'], axis = 1).drop(['Defense'], axis = 1).drop(['Sp. Atk'], axis = 1).drop(['Sp. Def'], axis = 1).drop(['Attack'], axis = 1).drop(['Legendary'], axis = 1)

fast, regular

In [ ]:
sns.boxplot(fast['Speed'])

In [ ]:
sns.boxplot(regular['Speed'])

In [ ]:
#нужно убрать выбросы
regular = regular.drop(regular.loc[regular['Speed'] > 150].index)

regular.info()

используем t-test так как распр. нормальное, а от выбросов избавились

In [ ]:
#H0 - в выборках нет стат значимых различий
#H1 - в выборках есть стат значимые различия

stat, p = st.ttest_ind(fast['Speed'], regular['Speed'], equal_var=False)
print('Statistics=%.3f, p=%.3f' % (stat, p))

alpha = 0.05
if p > alpha:
    print('нет стат значимых различий (не отвергаем H0)')
else:
    print('есть стат значимые различия (отвергаем H0)')

In [ ]:
if fast['Speed'].mean() > regular['Speed'].mean():
  print('Покемоны под наркотой быстрее других покемонов')

else:
  print('Покемоны под наркотой не быстрее других покемонов')

### Задание № 4:

Профессор Оук всегда любил истории про легендарных покемонов. Однако, профессор не очень уверен, что они самые лучшие относительно остальных покемонов. Оук предложил разобраться в этом нам. 

Проверьте, действительно ли сумма характеристик HP, Attack, Defense у легендарных покемонов выше, чем у других покемонов? 

А произведение этих же параметров? 

Найдите ответы на эти вопросы и убедите всех в своем выводе статистически.

Вы можете предположить, что распределение сум и произведений этих параметров является нормальным для всех классов покемонов.

In [ ]:
df_leg = pokemon.copy()
df_reg = pokemon.copy()

df_leg = df_leg[df_leg['Legendary'] == True]
df_reg = df_reg[df_reg['Legendary'] == False]

In [ ]:
df_leg = df_leg.drop(['pid'], axis = 1).drop(['Class 1'], axis = 1).drop(['Class 2'], axis = 1).drop(['Sp. Atk'], axis = 1).drop(['Sp. Def'], axis = 1).drop(['Speed'], axis = 1)

df_leg

In [ ]:
df_reg = df_reg.drop(['pid'], axis = 1).drop(['Class 1'], axis = 1).drop(['Class 2'], axis = 1).drop(['Sp. Atk'], axis = 1).drop(['Sp. Def'], axis = 1).drop(['Speed'], axis = 1)

df_reg

H0 = выборки показателей Attack для легендарных и нелегендарных покемонов не имеют статистически значимых различий

H1 = эти выборки статистически значимо различаются

In [ ]:
result = st.ttest_ind(df_leg['Attack'], df_reg['Attack'], equal_var=False)
alpha = 0.05

if (result.pvalue < alpha):
    print('Отвергаем нулевую гипотезу')
else:
    print('Не отвергаем нулевую гипотезу')

H0 = выборки показателей HP для легендарных и нелегендарных покемонов не имеют статистически значимых различий

H1 = эти выборки статистически значимо различаются

In [ ]:
result = st.ttest_ind(df_leg['HP'], df_reg['HP'], equal_var=False)
alpha = 0.05

if (result.pvalue < alpha):
    print('Отвергаем нулевую гипотезу')
else:
    print('Не отвергаем нулевую гипотезу')

H0 = выборки показателей Defense для легендарных и нелегендарных покемонов не имеют статистически значимых различий

H1 = эти выборки статистически значимо различаются

In [ ]:
result = st.ttest_ind(df_leg['Defense'], df_reg['Defense'], equal_var=False)
alpha = 0.05

if (result.pvalue < alpha):
    print('Отвергаем нулевую гипотезу')
else:
    print('Не отвергаем нулевую гипотезу')

всё окей, можем делать расчёты и сравнивать:

- Проверьте, действительно ли сумма характеристик HP, Attack, Defense у легендарных покемонов выше, чем у других покемонов?


In [ ]:
df_leg['Summa'] = df_leg.apply(lambda line: line['Attack'] + line['HP'] + line['Defense'], axis = 1)
df_reg['Summa'] = df_reg.apply(lambda line: line['Attack'] + line['HP'] + line['Defense'], axis = 1)

In [ ]:
if df_leg['Summa'].mean() > df_reg['Summa'].mean():
  print('Среднее суммы характеристик HP, Attack, Defense у легендарных покемонов выше, чем у других покемонов')
else:
  print('Среднее суммы характеристик HP, Attack, Defense у легендарных покемонов меньше, чем у других покемонов')

- Проверьте, действительно ли произведение характеристик HP, Attack, Defense у легендарных покемонов выше, чем у других покемонов?

In [ ]:
df_leg['Mult'] = df_leg.apply(lambda line: line['Attack'] * line['HP'] * line['Defense'], axis = 1)
df_reg['Mult'] = df_reg.apply(lambda line: line['Attack'] * line['HP'] * line['Defense'], axis = 1)

In [ ]:
if df_leg['Mult'].mean() > df_reg['Mult'].mean():
  print('Среднее произведения характеристик HP, Attack, Defense у легендарных покемонов выше, чем у других покемонов')
else:
  print('Среднее произведения характеристик HP, Attack, Defense у легендарных покемонов меньше, чем у других покемонов')

### Задание № 5:
Профессор Оук частенько наблюдает за боями покемонов. После очередных таких боев Оук выделил три класса best_defence_class, которые, на его взгляд, одинаковы по "силе обычной защиты" Defense. 

Проверьте, действительно ли эти классы покемонов не отличаются по уровню защиты статистически значимо? Все та же статистика вам в помощь!

Вы можете предположить, что распределение сум и произведений этих параметров является нормальным для всех классов покемонов.

<b><i>Это точно верная формулировка для целей задачи?</i></b>

- Вы можете предположить, что распределение сум и произведений этих параметров является нормальным для всех классов покемонов.

<b><i>Зачем в этой задаче распределение сумм и произведений, если мы сравниваем классы покемонов по Defense?</i></b>

<b><i>О каких параметрах речь?</i></b>

In [ ]:
best_defence_class = ['Rock', 'Ground','Steel','Ice']

In [ ]:
df_def = pokemon.copy()
df_def = df_def.drop(['pid'], axis = 1).drop(['HP'], axis = 1).drop(['Attack'], axis = 1).drop(['Sp. Atk'], axis = 1).drop(['Sp. Def'], axis = 1).drop(['Speed'], axis = 1).drop(['Legendary'], axis = 1)
df_def = df_def.loc[df_def['Class 1'].isin(['Rock', 'Ground','Steel','Ice']) | df_def['Class 2'].isin(['Rock', 'Ground','Steel','Ice'])]

df_def

In [ ]:
rock = df_def.loc[df_def['Class 1'].isin(['Rock']) | df_def['Class 2'].isin(['Rock'])]
ground = df_def.loc[df_def['Class 1'].isin(['Ground']) | df_def['Class 2'].isin(['Ground'])]
steel = df_def.loc[df_def['Class 1'].isin(['Steel']) | df_def['Class 2'].isin(['Steel'])]
ice = df_def.loc[df_def['Class 1'].isin(['Ice']) | df_def['Class 2'].isin(['Ice'])]

H0 = выбранные классы не отличаются по показателю Defense

H1 = выбранные классы отличаются по показателю Defense

In [ ]:
f, p = f_oneway(rock['Defense'], ground['Defense'], steel['Defense'], ice['Defense'])

alpha = 0.05
print(f, p)

if p <= alpha:
    print('Отклоняем H0')
else:
    print('Не отклоняем H0')